# H + COOH Coverage

#### Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1)

#### Setup

In [7]:
# Size of the surface
dim_x, dim_y = 100, 100

# Metals
metals = ['Ag', 'Au', 'Cu', 'Pd', 'Pt']

# Other parameters (Coming)

#### Run the simulation

In [34]:
surface = initialize_surface(dim_x, dim_y) # I should combine this with the occupance matrix in a dictionary

# Set a voltage
for voltage in voltage_sweep(start = 0.025, end = 1.05, scan_rate = 0.010): # Approximates the desired scan rate
    print(f"Voltage: {voltage}")
# Look through all sites 

# and predict all adsorbtion energies

# Log stats of the surface

Voltage: 0.025
Voltage: 0.03514851485148515
Voltage: 0.0452970297029703
Voltage: 0.05544554455445545
Voltage: 0.0655940594059406
Voltage: 0.07574257425742575
Voltage: 0.08589108910891091
Voltage: 0.09603960396039604
Voltage: 0.1061881188118812
Voltage: 0.11633663366336636
Voltage: 0.12648514851485151
Voltage: 0.13663366336633664
Voltage: 0.1467821782178218
Voltage: 0.15693069306930696
Voltage: 0.1670792079207921
Voltage: 0.17722772277227725
Voltage: 0.1873762376237624
Voltage: 0.19752475247524756
Voltage: 0.20767326732673272
Voltage: 0.21782178217821785
Voltage: 0.227970297029703
Voltage: 0.23811881188118816
Voltage: 0.2482673267326733
Voltage: 0.2584158415841585
Voltage: 0.26856435643564364
Voltage: 0.2787128712871288
Voltage: 0.28886138613861395
Voltage: 0.2990099009900991
Voltage: 0.3091584158415842
Voltage: 0.31930693069306937
Voltage: 0.3294554455445545
Voltage: 0.3396039603960397
Voltage: 0.34975247524752484
Voltage: 0.35990099009901
Voltage: 0.37004950495049516
Voltage: 0.380198

In [17]:
surface["adsorbants"]

array([['empty', 'empty', 'empty', ..., 'empty', 'empty', 'empty'],
       ['empty', 'empty', 'empty', ..., 'empty', 'empty', 'empty'],
       ['empty', 'empty', 'empty', ..., 'empty', 'empty', 'empty'],
       ...,
       ['empty', 'empty', 'empty', ..., 'empty', 'empty', 'empty'],
       ['empty', 'empty', 'empty', ..., 'empty', 'empty', 'empty'],
       ['empty', 'empty', 'empty', ..., 'empty', 'empty', 'empty']],
      dtype='<U5')

#### Plot the statistics

In [ ]:
# Make a nice plot showing how many neighbours, etc. there are at each voltage

#### Functions (To be moved to seperate .py file)

In [31]:
def initialize_surface(dim_x, dim_y): #Is still random - could be used with a seed in the name of reproduceability
    dim_z = 3
    surface_list = np.array([int(dim_x*dim_y*dim_z/len(metals))*[metals[metal_number]] for metal_number in range(len(metals))]).flatten() #Jack had a way shorter way of doing this, but I think it was random drawing instead of ensuring a perfectly even split
    np.random.shuffle(surface_list) #Shuffle list
    surf_atoms = np.reshape(surface_list, (dim_x, dim_y, dim_z)) #Reshape list to the
    surf_ads_ontop = np.reshape(["empty"]*dim_x*dim_y, (dim_x, dim_y))
    surf_ads_hollow = np.reshape(["empty"]*dim_x*dim_y, (dim_x, dim_y))
    surf = {"atoms": surf_atoms, "ads_top": surf_ads_ontop, "ads_hollow": surf_ads_ontop}
    return surface

def voltage_sweep(start, end, scan_rate):
    return np.linspace(start, end, int(np.abs(start - end) / scan_rate))

#def voltage_circle

#def voltage_up_down (All these could be worked into a single function)

array([0.025  , 0.28125, 0.5375 , 0.79375, 1.05   ])

In [30]:
np.abs(start - end) / number of numbers = scan_rate

0.20500000000000002